In [16]:
import numpy as np
import pandas as pd
import os
import glob
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

eps = np.finfo(float).eps

plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

import sys
sys.path.append('../')

import torch
import torch.nn as nn

from torchvision import transforms, utils, datasets
from torch.utils.data import  DataLoader, Dataset, TensorDataset

import torchvision.utils as vutils
import torch.nn.functional as F

from pyvacy import analysis
from sampler import *
from optimizer import DPSGD

In [5]:
use_cuda = True
if use_cuda:
    dtype = torch.cuda.FloatTensor
    device = torch.device("cuda")
    torch.cuda.set_device(0)
    print('GPU')
else:
    dtype = torch.FloatTensor
    device = torch.device("cpu")

GPU


In [6]:
image_transform = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST('data/mnist',train=True, download=True, transform=image_transform)
test_dataset = datasets.MNIST('data/mnist',train=False, download=True, transform=image_transform)

In [7]:
class Flatten(nn.Module):
    def forward(self, inp):
        return inp.reshape(inp.shape[0], -1)

class Classifier(nn.Module):
    def __init__(self, input_dim):
        super(Classifier, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 16, 8, 2),
            nn.ReLU(),
            nn.MaxPool2d(2, 1),
            nn.Conv2d(16, 32, 4, 2),
            nn.ReLU(),
            nn.MaxPool2d(2, 1),
            Flatten(),
            nn.Linear(288, 10),
            nn.LogSoftmax(dim=1))

    def forward(self, x):
        return self.net(x)



In [21]:
classifer = Classifier( input_dim=np.prod(train_dataset[0][0].shape)).to(device)

l2_norm_clip = 1.5 # gradient clip
noise_multiplier = 1.1 #random noise added
microbatch_size = 1
minibatch_size = 26
lr = 0.15
weight_decay = 0.001

optimizer = DPSGD(l2_norm_clip = l2_norm_clip, noise_multiplier = noise_multiplier, 
                        minibatch_size = minibatch_size, microbatch_size = microbatch_size, 
                        params = classifer.parameters(), lr = lr, weight_decay = weight_decay)

In [24]:
def train(model, optimizer, minibatch_size, microbatch_size, num_itr, device):

    loss_function = nn.NLLLoss()

    minibatch_loader, microbatch_loader = get_data_loaders(minibatch_size, microbatch_size, num_itr)

    iteration = 0
    for X_minibatch, y_minibatch in minibatch_loader(train_dataset):
        optimizer.zero_grad()
        for X_microbatch, y_microbatch in microbatch_loader(TensorDataset(X_minibatch, y_minibatch)):
            X_microbatch = X_microbatch.to(device)
            y_microbatch = y_microbatch.to(device)

            optimizer.zero_microbatch_grad()
            y_hat_microbatch = model(X_microbatch)
            loss = loss_function(y_hat_microbatch, y_microbatch)
            loss.backward()
            optimizer.microbatch_step()
        optimizer.step()

        if iteration % 10 == 0:
            print('[Iteration %d/%d] [Loss: %f]' % (iteration, num_itr, loss.item()))
        iteration += 1

    return model


In [ ]:

classifier = train(classifer, optimizer, minibatch_size, microbatch_size, 14000, device)

[Iteration 0/14000] [Loss: 0.002892]
[Iteration 10/14000] [Loss: 0.005150]
[Iteration 20/14000] [Loss: 0.012867]
[Iteration 30/14000] [Loss: 0.135098]
[Iteration 40/14000] [Loss: 1.237598]
[Iteration 50/14000] [Loss: 0.198805]
[Iteration 60/14000] [Loss: 0.059972]
[Iteration 70/14000] [Loss: 0.052361]
[Iteration 80/14000] [Loss: 0.008646]
[Iteration 90/14000] [Loss: 0.018140]
[Iteration 100/14000] [Loss: 0.014385]
[Iteration 110/14000] [Loss: 7.042806]
[Iteration 120/14000] [Loss: 7.076878]
[Iteration 130/14000] [Loss: 0.707346]
[Iteration 140/14000] [Loss: 0.003904]
[Iteration 150/14000] [Loss: 0.058160]
[Iteration 160/14000] [Loss: 0.005450]
[Iteration 170/14000] [Loss: 0.003870]
[Iteration 180/14000] [Loss: 4.042059]
[Iteration 190/14000] [Loss: 0.640369]
[Iteration 200/14000] [Loss: 11.004584]
[Iteration 210/14000] [Loss: 0.001165]
[Iteration 220/14000] [Loss: 0.080393]
[Iteration 230/14000] [Loss: 0.171749]
[Iteration 240/14000] [Loss: 0.000008]
[Iteration 250/14000] [Loss: 0.0023

In [29]:
test_dataloader = DataLoader(test_dataset, batch_size=len(test_dataset))
x, y = test_dataloader.__iter__().__next__()
x = x.to(device)
y = y.to(device)

y_pred = classifier(x).max(1)[1]

count = 0.
correct = 0.
for pred, actual in zip(y_pred, y):
    if pred.item() == actual.item():
        correct += 1.
    count += 1.
print('Test Accuracy: {}'.format(correct / count))

NameError: name 'classifier' is not defined